In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!mkdir /content/gdrive/My\ Drive/chk
!mkdir /content/gdrive/My\ Drive/cache

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
mkdir: cannot create directory ‘/content/gdrive/My Drive/chk’: File exists
mkdir: cannot create directory ‘/content/gdrive/My Drive/cache’: File exists


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import torch

SEED = 43
np.random.seed(SEED)
torch.manual_seed(SEED)

In [0]:
nrows = None
columns = ["text", "parent_text", "score"]
dtype_dict={'text': str, 'parent_text': str, 'score': int}
df = pd.concat([
    pd.read_csv("/content/gdrive/My Drive/comments_positive.csv", usecols=columns, dtype=dtype_dict, na_filter=False, nrows=nrows),
    pd.read_csv("/content/gdrive/My Drive/comments_negative.csv", usecols=columns, dtype=dtype_dict, na_filter=False, nrows=nrows)
], ignore_index=True)

In [0]:
df

,text,score,parent_text
0,This isn't Twitter: try to comment on the arti...,9582,Fucking faggot.
1,"Well, it is exactly what it sounds like. It's ...",9531,"Elaborate on this cum box, please."
2,"In soviet Russia, bomb disarms you!",8545,"I don't live in Russia anymore, and I will not..."
3,"""runin for senitur! #YOLO!""",7430,This just made me realize that future presiden...
4,You step motherfucker.,7173,I have sex with my step mom when my dad isn't ...
...,...,...,...
3999995,"""*THIS IS LIKE PRESIDENT OBAMA SAYING HE'S WOR...",-6,Even the Prime Minister Kevin Rudd is [concern...
3999996,No. Privilege is bunk. \n\nThis is strictly fo...,-6,So you believe in privilege when it affects yo...
3999997,how original,-6,Cue comments about Miami fans leaving
3999998,Don't just go spewing things like this w/o sou...,-6,The chief torturer of CIA is now Vice Presiden...


In [0]:
# It's more convenient to have scaled scores. To avoid leak during scaling I had to change train/test split a bit.
df_train, df_test = train_test_split(df, test_size=0.05, random_state=SEED)

target_scaler = StandardScaler()
df_train['scaled_score'] = target_scaler.fit_transform(df_train[['score']].values).flatten()
df_test['scaled_score'] = target_scaler.transform(df_test[['score']].values).flatten()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [0]:
df_train

,text,score,parent_text,scaled_score
155603,"I was a whole bunch of folks on Recess, yup. :...",454,Your IMDB just blew my mind. You were Upside D...,1.709493
1809639,"I'm not quite sure how that was relevant, but ...",71,http://i.imgur.com/GKLI7.jpg,-0.098607
3321125,Sounds like you weren't paying attention.,-9,Sadly high school history doesn't teach you ab...,-0.476278
2830504,&gt;Polygamy doesn't mean that there's just wo...,-12,&gt; from a feminist perspective \n\nSo what i...,-0.490440
1345396,The New Orleans Gargle-Glub-Glubs.,91,My high school's mascot was water...,-0.004189
...,...,...,...,...
2484501,Bacne/back-nee/backnee.,-16,Well formed.,-0.509324
1613617,Just hurl it against the bars and watch as it ...,78,"that phone looks like a nokia, you'll break ou...",-0.065561
2869503,I bet you'd be SO proud if your mom's tits wer...,-11,I think it's more offensive that she's suggest...,-0.485720
3296576,Supporting Libertarians is Supporting Satan.,-9,"EPS was never about hating on RP, not really. ...",-0.476278


In [0]:
y_train = df_train['scaled_score'].copy()
y_test = df_test['scaled_score'].copy()
df_train.drop(columns=['score', 'scaled_score'], inplace=True)
df_test.drop(columns=['score', 'scaled_score'], inplace=True)
X_train = df_train
X_test = df_test

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
# To be sure we don't use indices to predict something
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

print("Train shape: {}".format(X_train.shape))
print("Test shape: {}".format(X_test.shape))

Train shape: (3800000, 2)
Test shape: (200000, 2)


### Simple baseline code

In [0]:
y_pred = np.full(y_test.shape, y_train.mean())

### Metric

In [0]:
mean_squared_error(y_test, y_pred)

0.985947408090056

# Add features

In [0]:
X_train.text = X_train.text.str.replace('[^\w\s]',' ')
X_test.text = X_test.text.str.replace('[^\w\s]',' ')
X_train.parent_text = X_train.parent_text.str.replace('[^\w\s]',' ')
X_test.parent_text = X_test.parent_text.str.replace('[^\w\s]',' ')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [0]:
X_train = '"' + X_train + '"'
X_test = '"' + X_test + '"'

Save data for training

In [0]:
pd.concat([X_train, y_train], axis=1).sample(n=100000).to_csv('./train.csv', index=False, sep='\t')
pd.concat([X_test, y_test], axis=1).to_csv('./valid.csv', index=False, sep='\t')

In [0]:
del X_train, X_test, df_train, df_test, df

# PyTorch Bag of words

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext.data import Field, BucketIterator, TabularDataset, Iterator, LabelField
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import optim
import tqdm
import torchtext

In [0]:
torch.cuda.is_available()

True

In [0]:
BATCH_SIZE = 400
MAX_VOCAB_SIZE = 50000
EMBEDDING_SIZE = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'

In [0]:
def add_bigrams(tokens):
    bigrams = [f'{current_word} {next_word}' for current_word, next_word in 
                          zip(tokens, tokens[1:])]
    tokens.extend(bigrams)
    return tokens

In [0]:
TEXT = Field(tokenize="spacy",
             preprocessing=add_bigrams,
             batch_first=True,
             include_lengths=True)
SCORE = LabelField(dtype=torch.float, 
                   use_vocab=False)

train_data, valid_data = TabularDataset.splits(
        path='./', train='train.csv', validation='valid.csv', 
        format='tsv', csv_reader_params={"escapechar": '&'},
        fields={
                'text': ('text', TEXT),
                'parent_text': ('parent_text', TEXT),
                'scaled_score': ('score', SCORE)
              }
)

In [0]:
TEXT.build_vocab(train_data, 
                 max_size=MAX_VOCAB_SIZE,
                 vectors=f'glove.6B.{EMBEDDING_SIZE}d',
                 unk_init=torch.Tensor.normal_,
                 vectors_cache='/content/gdrive/My Drive/cache')

TEXT.vocab.vectors[TEXT.vocab.stoi['<unk>']] = 0
TEXT.vocab.vectors[TEXT.vocab.stoi['<pad>']] = 0

In [0]:
train_iter = BucketIterator(
        train_data,
        batch_size=BATCH_SIZE,
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        device=device
)

valid_iter = BucketIterator(
        valid_data,
        batch_size=BATCH_SIZE,
        sort=False,
        device=device
)

In [0]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0    
    model.train()
    
    for batch in tqdm.tqdm_notebook(iterator):
        optimizer.zero_grad()
        predictions = model(batch.text[0], batch.text[1], batch.parent_text[0], batch.parent_text[1])
        loss = criterion(predictions, batch.score)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text[0], batch.text[1], batch.parent_text[0], batch.parent_text[1])
            loss = criterion(predictions, batch.score)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

def get_preds(model, iterator):
    predictions = []
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            batch_predictions = model(batch.text[0], batch.text[1], batch.parent_text[0], batch.parent_text[1])
            predictions.append(batch_predictions.cpu().numpy())
    return np.array(predictions).flatten()

In [0]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
    
def train_loop(model, n_epochs, init_lr, decay_rate=0.1, decay_steps=1, save_name=None):
    optimizer = optim.Adam(model.parameters(), lr=init_lr)
    criterion = nn.MSELoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, decay_steps, decay_rate) 

    best_valid_loss = float('inf')

    for epoch in range(n_epochs):    
        start_time = time.time()
        
        train_loss = train(model, train_iter, optimizer, criterion)
        valid_loss = evaluate(model, valid_iter, criterion)
        
        scheduler.step()
        
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        if valid_loss < best_valid_loss and save_name is not None:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), save_name)
        
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

In [0]:
class EmbeddingBagModel(nn.Module):
    def __init__(self, 
                 vocab_size: int, 
                 embedding_dim: int,
                 text_fc_size: int, 
                 parent_fc_size: int,
                 padding_idx: int):
        super().__init__()
        self.padding_idx = padding_idx
        # EmbeddingBag lacks padding_idx so we have to use if in forward
        self.emb_bag = nn.EmbeddingBag(vocab_size, embedding_dim, mode='sum')
        self.text_emb_bn = nn.BatchNorm1d(embedding_dim)
        self.parent_emb_bn = nn.BatchNorm1d(embedding_dim)

        self.text_fc = nn.Linear(embedding_dim, text_fc_size)
        self.parent_fc = nn.Linear(embedding_dim, parent_fc_size)

        self.output_fc = nn.Linear(text_fc_size + parent_fc_size, 1)
        
    def init_emb_from_vocab(self, vocab: torchtext.vocab.Vocab, freeze=False):
        if not hasattr(vocab, 'vectors'):
            raise ValueError("To init embedding from vocab it needs to contain pretrained vectors")
        self.emb_bag.weight.data.copy_(vocab.vectors)
        if freeze:
            self.freeze_embedding()
        else:
            self.unfreeze_embedding()
        
    def forward(self, text, text_lengths, parent_text, parent_lengths):
        self.emb_bag.weight.data[self.padding_idx] = 0
        text = self.emb_bag(text) / text_lengths.view(-1, 1)
        text = self.text_emb_bn(text)
        text = F.relu(self.text_fc(text))

        parent_text = self.emb_bag(parent_text) / parent_lengths.view(-1, 1)
        parent_text = self.parent_emb_bn(parent_text)
        parent_text = F.relu(self.parent_fc(parent_text))

        joint = torch.cat([text, parent_text], dim=1)
        return self.output_fc(joint).squeeze(1)

    def freeze_embedding(self):
      self.emb_bag.weight.requires_grad = False

    def unfreeze_embedding(self):
      self.emb_bag.weight.requires_grad = True

In [0]:
TEXT_FC_SIZE = 200
PARENT_FC_SIZE = 200

model = EmbeddingBagModel(len(TEXT.vocab), EMBEDDING_SIZE, TEXT_FC_SIZE, PARENT_FC_SIZE, TEXT.vocab.stoi['<pad>'])
model.init_emb_from_vocab(TEXT.vocab, freeze=True)

model = model.to(device)

In [0]:
train_loop(model, 10, 1e-2, decay_rate=0.5, decay_steps=3, save_name='/content/gdrive/My Drive/chk/emb_bag.pt')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



Epoch: 01 | Epoch Time: 2m 41s
	Train Loss: 0.995 | Val. Loss: 0.980



Epoch: 02 | Epoch Time: 2m 38s
	Train Loss: 1.005 | Val. Loss: 0.988



Epoch: 03 | Epoch Time: 2m 37s
	Train Loss: 0.999 | Val. Loss: 0.966



Epoch: 04 | Epoch Time: 2m 35s
	Train Loss: 0.998 | Val. Loss: 0.968


...


Epoch: 06 | Epoch Time: 2m 37s
	Train Loss: 0.983 | Val. Loss: 0.976



Epoch: 07 | Epoch Time: 2m 38s
	Train Loss: 0.976 | Val. Loss: 0.975



Epoch: 08 | Epoch Time: 2m 38s
	Train Loss: 0.972 | Val. Loss: 0.976



Epoch: 09 | Epoch Time: 2m 41s
	Train Loss: 0.968 | Val. Loss: 0.972



Epoch: 10 | Epoch Time: 2m 38s
	Train Loss: 0.962 | Val. Loss: 0.975


In [0]:
model.unfreeze_embedding()
train_loop(model, 10, 1e-2, decay_rate=0.5, save_name='/content/gdrive/My Drive/chk/emb_bag.pt')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



Epoch: 01 | Epoch Time: 2m 46s
	Train Loss: 0.965 | Val. Loss: 0.937



Epoch: 02 | Epoch Time: 2m 48s
	Train Loss: 0.805 | Val. Loss: 0.993



Epoch: 03 | Epoch Time: 2m 47s
	Train Loss: 0.571 | Val. Loss: 1.041



Epoch: 04 | Epoch Time: 2m 45s
	Train Loss: 0.410 | Val. Loss: 1.088



Epoch: 05 | Epoch Time: 2m 51s
	Train Loss: 0.345 | Val. Loss: 1.105



Epoch: 06 | Epoch Time: 2m 45s
	Train Loss: 0.317 | Val. Loss: 1.123



Epoch: 07 | Epoch Time: 2m 47s
	Train Loss: 0.305 | Val. Loss: 1.112


KeyboardInterrupt: ignored

In [0]:
mean_squared_error(y_test, get_preds(model, valid_iter))

1.3064179956017774

In [0]:
BATCH_SIZE = 400
MAX_VOCAB_SIZE = 20000
EMBEDDING_SIZE = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'

In [0]:
def get_min_length_preprocessor(length):
    def pad_to_min_length(tokens):
        if len(tokens) < length:
            tokens.extend(['<pad>'] * (length - len(tokens)))
        return tokens
    return pad_to_min_length

TEXT = Field(tokenize="spacy",
             preprocessing=get_min_length_preprocessor(5),
             batch_first=True,
             include_lengths=True)
SCORE = LabelField(dtype=torch.float, 
                   use_vocab=False)

train_data, valid_data = TabularDataset.splits(
        path='./', train='train.csv', validation='valid.csv', 
        format='tsv', csv_reader_params={"escapechar": '&'},
        fields={
                'text': ('text', TEXT),
                'parent_text': ('parent_text', TEXT),
                'scaled_score': ('score', SCORE)
              }
)

TEXT.build_vocab(train_data, 
                 max_size=MAX_VOCAB_SIZE,
                 vectors=f'glove.6B.{EMBEDDING_SIZE}d',
                 unk_init=torch.Tensor.normal_,
                 vectors_cache='/content/gdrive/My Drive/cache')

TEXT.vocab.vectors[TEXT.vocab.stoi['<unk>']] = 0
TEXT.vocab.vectors[TEXT.vocab.stoi['<pad>']] = 0

train_iter = BucketIterator(
        train_data,
        batch_size=BATCH_SIZE,
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        device=device
)

valid_iter = BucketIterator(
        valid_data,
        batch_size=BATCH_SIZE,
        sort=False,
        device=device
)

In [0]:
class CNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, hidden_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.conv_0 = nn.Conv2d(in_channels=1,
                                out_channels=n_filters, 
                                kernel_size=(filter_sizes[0], embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, 
                                out_channels=n_filters, 
                                kernel_size=(filter_sizes[1], embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, 
                                out_channels=n_filters, 
                                kernel_size=(filter_sizes[2], embedding_dim))
        self.fc = nn.Linear(2 * len(filter_sizes) * n_filters, hidden_dim)
        self.output_fc = nn.Linear(hidden_dim, 1)
        
        self.dropout = nn.Dropout(0.5)

    def init_emb_from_vocab(self, vocab: torchtext.vocab.Vocab, freeze=False):
        if not hasattr(vocab, 'vectors'):
            raise ValueError("To init embedding from vocab it needs to contain pretrained vectors")
        self.embedding.weight.data.copy_(vocab.vectors)
        if freeze:
            self.freeze_embedding()
        else:
            self.unfreeze_embedding()

    def forward(self, text, text_lengths, parent_text, parent_lengths):
        text = self.embedding(text).unsqueeze(1)
        text_conv0 = F.relu(self.conv_0(text).squeeze(3))
        text_conv1 = F.relu(self.conv_1(text).squeeze(3))
        text_conv2 = F.relu(self.conv_2(text).squeeze(3))

        text_pooled0 = F.max_pool1d(text_conv0, text_conv0.shape[2]).squeeze(2)
        text_pooled1 = F.max_pool1d(text_conv1, text_conv1.shape[2]).squeeze(2)
        text_pooled2 = F.max_pool1d(text_conv2, text_conv2.shape[2]).squeeze(2)
        
        parent_text = self.embedding(parent_text).unsqueeze(1)
        parent_conv0 = F.relu(self.conv_0(parent_text).squeeze(3))
        parent_conv1 = F.relu(self.conv_1(parent_text).squeeze(3))
        parent_conv2 = F.relu(self.conv_2(parent_text).squeeze(3))

        parent_pooled0 = F.max_pool1d(parent_conv0, parent_conv0.shape[2]).squeeze(2)
        parent_pooled1 = F.max_pool1d(parent_conv1, parent_conv1.shape[2]).squeeze(2)
        parent_pooled2 = F.max_pool1d(parent_conv2, parent_conv2.shape[2]).squeeze(2)

        joint = self.dropout(torch.cat((text_pooled0, text_pooled1, text_pooled2,
                                        parent_pooled0, parent_pooled1, parent_pooled2), dim = 1))
        joint = F.relu(self.fc(joint))
        return self.output_fc(joint).squeeze(1)
        
    def freeze_embedding(self):
      self.embedding.weight.requires_grad = False

    def unfreeze_embedding(self):
      self.embedding.weight.requires_grad = True

In [0]:
JOINT_SIZE = 300
N_FILTERS = 30
FILTER_SIZES = [3,4,5]

model = CNNModel(len(TEXT.vocab), EMBEDDING_SIZE, N_FILTERS, FILTER_SIZES, JOINT_SIZE, TEXT.vocab.stoi['<pad>'])
model.init_emb_from_vocab(TEXT.vocab, freeze=True)

model = model.to(device)

In [0]:
train_loop(model, 20, 1e-2, decay_rate=0.3, decay_steps=4, save_name='/content/gdrive/My Drive/chk/cnn.pt')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



Epoch: 01 | Epoch Time: 1m 36s
	Train Loss: 1.131 | Val. Loss: 0.982



Epoch: 02 | Epoch Time: 1m 34s
	Train Loss: 1.016 | Val. Loss: 0.978



Epoch: 03 | Epoch Time: 1m 36s
	Train Loss: 1.009 | Val. Loss: 0.965



Epoch: 04 | Epoch Time: 1m 36s
	Train Loss: 0.998 | Val. Loss: 0.961



Epoch: 05 | Epoch Time: 1m 35s
	Train Loss: 0.983 | Val. Loss: 0.944



Epoch: 06 | Epoch Time: 1m 35s
	Train Loss: 0.977 | Val. Loss: 0.942


In [0]:
model.unfreeze_embedding()
train_loop(model, 10, 1e-3, decay_rate=0.5, decay_steps=3, save_name='/content/gdrive/My Drive/chk/cnn.pt')

In [0]:
train_loop(model, 15, 1e-3, decay_rate=0.5, decay_steps=5, save_name='/content/gdrive/My Drive/chk/cnn.pt')

In [0]:
mean_squared_error(y_test, get_preds(model, valid_iter))